In [34]:
import polars as pl
import os
import numpy as np
import torch, torchvision
from torchvision.utils import make_grid
from glio.loaders import nifti
from glio.python_tools import get_all_files
from glio.jupyter_tools import clean_mem, show_slices_arr, show_slices
from glio.visualize import Visualizer, vis_imshow
import matplotlib.pyplot as plt

pl.Config.set_fmt_str_lengths(900)
pl.Config.set_tbl_width_chars(900)

path = r"E:\dataset\LUMIERE"

# LUMIERE
## Метаданные

In [159]:
demographics_pathology = pl.read_csv(f"{path}/LUMIERE-Demographics_Pathology.csv")
demographics_pathology

Patient,Survival time (weeks),Sex,Age at surgery (years),IDH (WT: wild type),IDH method,MGMT qualitative,MGMT quantitative
str,str,str,i64,str,str,str,str
"""Patient-001""","""89""","""female""",57,"""WT""","""BES""","""methylated""","""na"""
"""Patient-002""","""48""","""female""",71,"""na""","""na""","""not methylated""","""0.00%"""
"""Patient-003""","""40""","""male""",39,"""WT""","""Immuno""","""not methylated""","""0.00%"""
"""Patient-004""","""110""","""male""",69,"""WT""","""Sanger Sequencing""","""not methylated""","""0.00%"""
"""Patient-005""","""53""","""male""",66,"""WT""","""Immuno""","""not methylated""","""0.00%"""
…,…,…,…,…,…,…,…
"""Patient-087""","""77""","""male""",64,"""na""","""na""","""not methylated""","""0.00%"""
"""Patient-088""","""58""","""male""",73,"""WT""","""Immuno""","""not methylated""","""0.00%"""
"""Patient-089""","""82""","""female""",57,"""WT""","""BES""","""methylated""","""na"""


In [160]:
expert_rating = pl.read_csv(f"{path}/LUMIERE-ExpertRating-v202211.csv")
expert_rating

Patient,Date,LessThan3Months,NonMeasurableLesions,"Rating (according to RANO, PD: Progressive disease, SD: Stable disease, PR: Partial response, CR: Complete response, Pre-Op: Pre-Operative, Post-Op: Post-Operative)","Rating rationale (CRET: complete resection of the enhancing tumor, PRET: partial resection of the enhancing tumor, T2-Progr.: T2-Progression, L: Lesion)"
str,str,str,str,str,str
"""Patient-001""","""week-000-1""",null,null,"""Pre-Op""",null
"""Patient-001""","""week-000-2""",null,null,"""Post-Op""","""CRET"""
"""Patient-001""","""week-044""",null,null,"""SD""",null
"""Patient-001""","""week-056""",null,"""x""","""PD""","""Target L.: 1 12mm x 13mm, new non-target L."""
"""Patient-002""","""week-000""",null,null,"""Pre-Op""",null
…,…,…,…,…,…
"""Patient-091""","""week-001""",null,null,"""Post-Op ""","""PRET"""
"""Patient-091""","""week-014""",null,"""x""","""PD""","""Non-measurable L."""
"""Patient-091""","""week-026""",null,"""x""","""PD""","""Non-measurable L."""


`RANO` - колонка, по которой можно фильтровать постоперационные изображения.

In [39]:
RANO = "Rating (according to RANO, PD: Progressive disease, SD: Stable disease, PR: Partial response, CR: Complete response, Pre-Op: Pre-Operative, Post-Op: Post-Operative)"
RANO_rationale = "Rating rationale (CRET: complete resection of the enhancing tumor, PRET: partial resection of the enhancing tumor, T2-Progr.: T2-Progression, L: Lesion)"
expert_rating.unique(RANO).select("Patient", "Date", RANO, RANO_rationale)

Patient,Date,"Rating (according to RANO, PD: Progressive disease, SD: Stable disease, PR: Partial response, CR: Complete response, Pre-Op: Pre-Operative, Post-Op: Post-Operative)","Rating rationale (CRET: complete resection of the enhancing tumor, PRET: partial resection of the enhancing tumor, T2-Progr.: T2-Progression, L: Lesion)"
str,str,str,str
"""Patient-004""","""week-071""","""PR""","""Regressing target L. 12mm x 20mm, regressing non-target L."""
"""Patient-001""","""week-056""","""PD""","""Target L.: 1 12mm x 13mm, new non-target L."""
"""Patient-026""","""week-069""","""None""","""None"""
"""Patient-001""","""week-000-2""","""Post-Op""","""CRET"""
"""Patient-009""","""week-024""","""CR""","""None"""
"""Patient-001""","""week-044""","""SD""",null
"""Patient-001""","""week-000-1""","""Pre-Op""",null
"""Patient-028""","""week-000-2""","""Post-Op ""","""CRET"""
"""Patient-089""","""week-046""","""Post-Op/PD""","""PRET, non-Target L."""


Метки:
- `PR` - partial response
- `PD` - progressive disease
- `CR` - complete response
- `SD` - stable disease
- `Post-Op`
- `Post-Op `
- `Post-Op/PD`

В итоге получается 425 обследования, 91 пациент

In [49]:
expert_rating.filter(pl.col(RANO).is_in(["PR", "PD", "CR", "CD", "Post-Op", "Post-Op ", "Post-Op/PD"]))

Patient,Date,LessThan3Months,NonMeasurableLesions,"Rating (according to RANO, PD: Progressive disease, SD: Stable disease, PR: Partial response, CR: Complete response, Pre-Op: Pre-Operative, Post-Op: Post-Operative)","Rating rationale (CRET: complete resection of the enhancing tumor, PRET: partial resection of the enhancing tumor, T2-Progr.: T2-Progression, L: Lesion)"
str,str,str,str,str,str
"""Patient-001""","""week-000-2""",null,null,"""Post-Op""","""CRET"""
"""Patient-001""","""week-056""",null,"""x""","""PD""","""Target L.: 1 12mm x 13mm, new non-target L."""
"""Patient-002""","""week-003""",null,null,"""Post-Op""","""CRET"""
"""Patient-002""","""week-037""",null,null,"""PD""","""Target L.: 13mm x 26mm"""
"""Patient-002""","""week-040-2""",null,null,"""Post-Op""","""None"""
…,…,…,…,…,…
"""Patient-091""","""week-001""",null,null,"""Post-Op ""","""PRET"""
"""Patient-091""","""week-014""",null,"""x""","""PD""","""Non-measurable L."""
"""Patient-091""","""week-026""",null,"""x""","""PD""","""Non-measurable L."""


In [59]:
# кол-во обследований каждого пациента
print(expert_rating.filter(pl.col(RANO).is_in(["PR", "PD", "CR", "CD", "Post-Op", "Post-Op ", "Post-Op/PD"])).to_series(0).unique_counts().to_list())

[2, 4, 3, 7, 1, 6, 9, 4, 5, 3, 3, 5, 2, 1, 13, 1, 2, 4, 9, 1, 3, 5, 11, 5, 4, 1, 2, 5, 13, 5, 16, 12, 8, 5, 4, 1, 4, 2, 2, 2, 3, 9, 7, 1, 2, 2, 1, 4, 3, 4, 8, 7, 1, 7, 7, 1, 1, 4, 11, 4, 5, 4, 5, 3, 6, 8, 15, 6, 1, 3, 7, 11, 7, 3, 3, 1, 6, 14, 2, 1, 2, 2, 1, 1, 5, 4, 3, 1, 6, 2, 5]


## DeepBraTumIA
Обследование `E:\dataset\LUMIERE\Imaging\Patient-004\week-071` - Partial response

In [63]:
patient = expert_rating.filter((pl.col("Patient") == "Patient-004") & (pl.col("Date") == "week-071"))
patient

Patient,Date,LessThan3Months,NonMeasurableLesions,"Rating (according to RANO, PD: Progressive disease, SD: Stable disease, PR: Partial response, CR: Complete response, Pre-Op: Pre-Operative, Post-Op: Post-Operative)","Rating rationale (CRET: complete resection of the enhancing tumor, PRET: partial resection of the enhancing tumor, T2-Progr.: T2-Progression, L: Lesion)"
str,str,str,str,str,str
"""Patient-004""","""week-071""",null,null,"""PR""","""Regressing target L. 12mm x 20mm, regressing non-target L."""


In [134]:
pl.Config.set_fmt_str_lengths(100)
deepbratumia = pl.read_csv(f"{path}/LUMIERE-pyradiomics-deepbratumia-features.csv")
patient = deepbratumia.filter((pl.col("Patient") == "Patient-004") & (pl.col("Time point") == "week-071")).select("Patient", "Time point", "Image", "Mask", "Label name", "Sequence", "diagnostics_Mask-original_BoundingBox")
patient

Patient,Time point,Image,Mask,Label name,Sequence,diagnostics_Mask-original_BoundingBox
str,str,str,str,str,str,str
"""Patient-004""","""week-071""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Necrosis""","""CT1""","""(31, 80, 57, 15, 17, 12)"""
"""Patient-004""","""week-071""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Contrast-enhancing""","""CT1""","""(31, 80, 57, 19, 49, 12)"""
"""Patient-004""","""week-071""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Edema""","""CT1""","""(32, 64, 57, 50, 106, 75)"""
"""Patient-004""","""week-071""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/skull_strip/t1_skull_strip.nii.gz""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Necrosis""","""T1""","""(31, 80, 57, 15, 17, 12)"""
"""Patient-004""","""week-071""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/skull_strip/t1_skull_strip.nii.gz""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Contrast-enhancing""","""T1""","""(31, 80, 57, 19, 49, 12)"""
…,…,…,…,…,…,…
"""Patient-004""","""week-071""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/skull_strip/t2_skull_strip.nii.gz""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Contrast-enhancing""","""T2""","""(31, 80, 57, 19, 49, 12)"""
"""Patient-004""","""week-071""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/skull_strip/t2_skull_strip.nii.gz""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Edema""","""T2""","""(32, 64, 57, 50, 106, 75)"""
"""Patient-004""","""week-071""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz""","""Patient-004/week-071/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Necrosis""","""FLAIR""","""(31, 80, 57, 15, 17, 12)"""


Пример изображения из LUMIERE-pyradiomics-deepbratumia-features.csv.
```txt
Label   Label name          diagnostics_Mask-original_BoundingBox
1       Necrosis            (31, 80, 57, 15, 17, 12)
2       Contrast-enhancing  (31, 80, 57, 19, 49, 12)
3       Edema               (32, 64, 57, 50, 106, 75)
```

In [60]:
observation = f"{path}/Imaging/Patient-004/week-071"
image_ct1 = nifti.read(f"{observation}/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz")
image_t1 = nifti.read(f"{observation}/DeepBraTumIA-segmentation/atlas/skull_strip/t1_skull_strip.nii.gz")
image_t2 = nifti.read(f"{observation}/DeepBraTumIA-segmentation/atlas/skull_strip/t2_skull_strip.nii.gz")
image_flair = nifti.read(f"{observation}/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz")
mask = nifti.read(f"{observation}/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz")
[i.shape for i in (image_ct1, image_t1, image_t2, image_flair, mask)]

[torch.Size([182, 218, 182]),
 torch.Size([182, 218, 182]),
 torch.Size([182, 218, 182]),
 torch.Size([182, 218, 182]),
 torch.Size([182, 218, 182])]

In [61]:
show_slices_arr((image_ct1, image_t1, image_t2, image_flair, mask))

interactive(children=(Checkbox(value=False, description='color'), Text(value='0 1 2 3', description='permute')…

<function glio.jupyter_tools.show_slices_arr.<locals>.f(color, permute: str, **kwargs)>

Bounding Box доступен в поле `diagnostics_Mask-original_BoundingBox`

Формат: `X1, Y1, Z1, X2-X1, Y2-Y1, Z2-Z1`

Извлечение целочисленных значений:

In [103]:
necrosis = [int(i) for i in patient[0]["diagnostics_Mask-original_BoundingBox"][0][1:-1].split(", ")]
contench = [int(i) for i in patient[1]["diagnostics_Mask-original_BoundingBox"][0][1:-1].split(", ")]# contrast_enchancing
edema = [int(i) for i in patient[2]["diagnostics_Mask-original_BoundingBox"][0][1:-1].split(", ")]
f"{necrosis = }, {contench = }, {edema = }"

'necrosis = [31, 80, 57, 15, 17, 12], contench = [31, 80, 57, 19, 49, 12], edema = [32, 64, 57, 50, 106, 75]'

In [106]:
bb_ct1 = torch.stack([image_ct1.detach().clone(),image_ct1.detach().clone(),image_ct1.detach().clone()])
val = 2
# Label   Label name          diagnostics_Mask-original_BoundingBox
# 1       Necrosis            (31, 80, 57, 15, 17, 12)
# 2       Contrast-enhancing  (31, 80, 57, 19, 49, 12)
# 3       Edema               (32, 64, 57, 50, 106, 75)
bb_ct1[0, necrosis[0]:necrosis[0]+necrosis[3], necrosis[1]:necrosis[1]+necrosis[4], necrosis[2]:necrosis[2]+necrosis[5]] /= 2
bb_ct1[1, contench[0]:contench[0]+contench[3], contench[1]:contench[1]+contench[4], contench[2]:contench[2]+contench[5]] /= 2
bb_ct1[2, edema[0]:edema[0]+edema[3], edema[1]:edema[1]+edema[4], edema[2]:edema[2]+edema[5]] /= 2

In [107]:
show_slices_arr(bb_ct1)

interactive(children=(Checkbox(value=False, description='color'), Text(value='0 1 2 3', description='permute')…

<function glio.jupyter_tools.show_slices_arr.<locals>.f(color, permute: str, **kwargs)>

### Native

In [108]:
native_ct1_mask = nifti.read(rf"{observation}\DeepBraTumIA-segmentation\native\segmentation\ct1_seg_mask.nii.gz")
native_ct1 = nifti.read(rf"{observation}\DeepBraTumIA-segmentation\native\skull_strip\ct1_brain_mask.nii.gz")

In [109]:
show_slices_arr([native_ct1,native_ct1,native_ct1_mask])

interactive(children=(Checkbox(value=False, description='color'), Text(value='0 1 2 3', description='permute')…

<function glio.jupyter_tools.show_slices_arr.<locals>.f(color, permute: str, **kwargs)>

## HD-GLIO-AUTO
Судя по всему сегментация более низкого качества.

Пример изображения из LUMIERE-pyradiomics-hdglioauto-features.csv.
```txt
Label   Label name          diagnostics_Mask-original_BoundingBox
1       Non-enhancing       (330, 115, 11, 141, 170, 9)
2       Contrast-enhancing  (344, 148, 11, 100, 110, 8)

```

In [168]:
hdglioauto = pl.read_csv(f"{path}/LUMIERE-pyradiomics-hdglioauto-features.csv", infer_schema_length=10000)
hdglioauto

Patient,Time point,Image,Mask,Label name,Label,Sequence,Reader,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,diagnostics_Mask-original_BoundingBox,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Mask-original_CenterOfMassIndex,diagnostics_Mask-original_CenterOfMass,diagnostics_Image-interpolated_Spacing,diagnostics_Image-interpolated_Size,diagnostics_Image-interpolated_Mean,diagnostics_Image-interpolated_Minimum,diagnostics_Image-interpolated_Maximum,diagnostics_Mask-interpolated_Spacing,diagnostics_Mask-interpolated_Size,…,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glrlm_ShortRunLowGrayLevelEmphasis,original_glszm_GrayLevelNonUniformity,original_glszm_GrayLevelNonUniformityNormalized,original_glszm_GrayLevelVariance,original_glszm_HighGrayLevelZoneEmphasis,original_glszm_LargeAreaEmphasis,original_glszm_LargeAreaHighGrayLevelEmphasis,original_glszm_LargeAreaLowGrayLevelEmphasis,original_glszm_LowGrayLevelZoneEmphasis,original_glszm_SizeZoneNonUniformity,original_glszm_SizeZoneNonUniformityNormalized,original_glszm_SmallAreaEmphasis,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_gldm_DependenceEntropy,original_gldm_DependenceNonUniformity,original_gldm_DependenceNonUniformityNormalized,original_gldm_DependenceVariance,original_gldm_GrayLevelNonUniformity,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,i64,i64,str,str,str,str,f64,f64,f64,str,str,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Patient-001""","""week-000-1""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/CT1_r2s_bet_reg.nii.gz""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/segmentation.nii.gz""","""Non-enhancing""",1,"""CT1""","""N-A""","""v3.0.1""","""1.21.5""","""2.0.2""","""1.1.1""","""3.7.7""","""{'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 100, 'remo…","""{'Original': {}}""","""0b0f0376e1deacd3c52c148ce6c3f094dd38dc1b""","""3D""","""(0.359375, 0.359375, 6.000002861022949)""","""(640, 640, 24)""",42.184499,-35.0,1114.0,"""eb5657918642675c47619264e3e7a2aec64ba1ec""","""(0.359375, 0.359375, 6.000002861022949)""","""(640, 640, 24)""","""(330, 115, 11, 141, 170, 9)""",49365,7,"""(392.75897903372834, 214.39226172389345, 14.216226071103009)""","""(-19.092916656509317, 17.434763007617633, -1.025096678094556)""","""(1.0, 1.0, 1.0)""","""(63, 73, 66)""",143.719037,-126.703893,921.56103,"""(1.0, 1.0, 1.0)""","""(63, 73, 66)""",…,1606.17723,0.000859,304.506911,0.025354,137.801451,1927.725479,95.991174,130684.393505,0.082614,0.001047,4568.817652,0.380418,0.640617,1301.099279,0.000693,7.699979,0.314579,85.886081,8.03657,6399.029336,

In [110]:
image_ct1 = nifti.read(f"{observation}/HD-GLIO-AUTO-segmentation/registered/CT1_r2s_bet_reg.nii.gz")
image_t1 = nifti.read(f"{observation}/HD-GLIO-AUTO-segmentation/registered/T1_r2s_bet_reg.nii.gz")
image_t2 = nifti.read(f"{observation}/HD-GLIO-AUTO-segmentation/registered/T2_r2s_bet_reg.nii.gz")
image_flair = nifti.read(f"{observation}/HD-GLIO-AUTO-segmentation/registered/FLAIR_r2s_bet_reg.nii.gz")
mask = nifti.read(f"{observation}/HD-GLIO-AUTO-segmentation/registered/segmentation.nii.gz")
[i.shape for i in (image_ct1, image_t1, image_t2, image_flair, mask)]

[torch.Size([160, 256, 256]),
 torch.Size([160, 256, 256]),
 torch.Size([160, 256, 256]),
 torch.Size([160, 256, 256]),
 torch.Size([160, 256, 256])]

In [111]:
show_slices_arr((image_ct1, image_t1, image_t2, image_flair, mask))

interactive(children=(Checkbox(value=False, description='color'), Text(value='0 1 2 3', description='permute')…

<function glio.jupyter_tools.show_slices_arr.<locals>.f(color, permute: str, **kwargs)>

In [11]:
bb_ct1 = torch.stack([image_ct1.detach().clone(),image_ct1.detach().clone(),image_ct1.detach().clone()])
val = 2
# Label   Label name          diagnostics_Mask-original_BoundingBox
# 1       Non-enhancing       (330, 115, 11, 141, 170, 9)
# 2       Contrast-enhancing  (344, 148, 11, 100, 110, 8)
bb_ct1[0, 330:330+141, 115:115+170, 11:11+9] /= 2
bb_ct1[1, 344:344+100, 148:148+110, 11:11+8] /= 2

In [12]:
show_slices_arr(bb_ct1)

interactive(children=(Checkbox(value=False, description='color'), Text(value='0 1 2 3', description='permute')…

<function glio.jupyter_tools.show_slices_arr.<locals>.f(color, permute: str, **kwargs)>

## Создание DS

In [186]:
demographics_pathology = pl.read_csv(
    f"{path}/LUMIERE-Demographics_Pathology.csv", infer_schema_length=10000
)


def rano_norm(x: str):
    if x == "Post-Op ":
        return "Post-Op"
    elif x == "Post-Op/PD":
        return "PD"
    else:
        return x


expert_rating = (
    pl.read_csv(f"{path}/LUMIERE-ExpertRating-v202211.csv", infer_schema_length=10000)
    .with_columns(RANO=pl.col(RANO).map_elements(rano_norm))
    .drop(RANO, RANO_rationale)
)

deepbratumia = (
    pl.read_csv(
        f"{path}/LUMIERE-pyradiomics-deepbratumia-features.csv",
        infer_schema_length=10000,
    )
    .select(
        "Patient",
        "Time point",
        "Image",
        "Mask",
        "Label name",
        "Sequence",
        "diagnostics_Mask-original_BoundingBox",
    )
    .with_columns(
        bbox=pl.col("diagnostics_Mask-original_BoundingBox").map_elements(
            lambda x: [int(i) for i in x[1:-1].split(", ")]
        ),
        src=pl.lit("Deepbratumia"),
    )
    .drop("diagnostics_Mask-original_BoundingBox")
    .rename({"Time point": "Date"})
)

hdglioauto = (
    pl.read_csv(
        f"{path}/LUMIERE-pyradiomics-hdglioauto-features.csv", infer_schema_length=10000
    )
    .select(
        "Patient",
        "Time point",
        "Image",
        "Mask",
        "Label name",
        "Sequence",
        "diagnostics_Mask-original_BoundingBox",
    )
    .with_columns(
        bbox=pl.col("diagnostics_Mask-original_BoundingBox").map_elements(
            lambda x: [int(i) for i in x[1:-1].split(", ")]
        ),
        src=pl.lit("HD-GLIO-AUTO"),
    )
    .drop("diagnostics_Mask-original_BoundingBox")
    .rename({"Time point": "Date"})
)

# - `PR` - partial response
# - `PD` - progressive disease
# - `CR` - complete response
# - `SD` - stable disease
# - `Post-Op`
# - `Post-Op `
# - `Post-Op/PD`

combined = (
    demographics_pathology.join(expert_rating, on="Patient", how="outer_coalesce")
    .join(deepbratumia, on=["Patient", "Date"], how="outer_coalesce")
    .join(hdglioauto, on=["Patient", "Date"], how="outer_coalesce")
)
combined


Patient,Survival time (weeks),Sex,Age at surgery (years),IDH (WT: wild type),IDH method,MGMT qualitative,MGMT quantitative,Date,LessThan3Months,NonMeasurableLesions,RANO,Image,Mask,Label name,Sequence,bbox,src,Image_right,Mask_right,Label name_right,Sequence_right,bbox_right,src_right
str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,str,str,list[i64],str
"""Patient-001""","""89""","""female""",57,"""WT""","""BES""","""methylated""","""na""","""week-000-1""",null,null,"""Pre-Op""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Necrosis""","""CT1""","[51, 39, … 45]","""Deepbratumia""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/CT1_r2s_bet_reg.nii.gz""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/segmentation.nii.gz""","""Non-enhancing""","""CT1""","[330, 115, … 9]","""HD-GLIO-AUTO"""
"""Patient-001""","""89""","""female""",57,"""WT""","""BES""","""methylated""","""na""","""week-000-1""",null,null,"""Pre-Op""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Necrosis""","""CT1""","[51, 39, … 45]","""Deepbratumia""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/CT1_r2s_bet_reg.nii.gz""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/segmentation.nii.gz""","""Contrast-enhancing""","""CT1""","[344, 148, … 8]","""HD-GLIO-AUTO"""
"""Patient-001""","""89""","""female""",57,"""WT""","""BES""","""methylated""","""na""","""week-000-1""",null,null,"""Pre-Op""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Necrosis""","""CT1""","[51, 39, … 45]","""Deepbratumia""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/T1_r2s_bet_reg.nii.gz""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/segmentation.nii.gz""","""Non-enhancing""","""T1""","[330, 115, … 9]","""HD-GLIO-AUTO"""
"""Patient-001""","""89""","""female""",57,"""WT""","""BES""","""methylated""","""na""","""week-000-1""",null,null,"""Pre-Op""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Necrosis""","""CT1""","[51, 39, … 45]","""Deepbratumia""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/T1_r2s_bet_reg.nii.gz""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/segmentation.nii.gz""","""Contrast-enhancing""","""T1""","[344, 148, … 8]","""HD-GLIO-AUTO"""
"""Patient-001""","""89""","""female""",57,"""WT""","""BES""","""methylated""","""na""","""week-000-1""",null,null,"""Pre-Op""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","""Necrosis""","""CT1""","[51, 39, … 45]","""Deepbratumia""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/T2_r2s_bet_reg.nii.gz""","""Imaging/Patient-001/week-000-1/HD-GLIO-AUTO-segmentation/registered/segmentation.nii.gz""","""Non-enhancing""","""T2""","[330, 115, … 9]","""HD-GLIO-AUTO"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Patient-025""","""85""","""male""",68,"""WT""","""Sanger Sequencing""","""not methylated""","""0.00%""","""week-002""",null,null,"""Post-Op""",null,null,null,null,null,null,null,null,null,null,null,null
"""Patient-026""","""na""","""male""",45,"""IDH1 neg, Sequencing required""","""Immuno""","""na""","""na""","""week-069""",null,null,"""None""",null,null,null,null,null,null,null,null

In [189]:
d = {}
for ob in combined.rows(named=True):
    name = f'{ob["Patient"]}/{ob["Date"]}'
    if name not in d: d[name] = ob.copy()
    if "Sequence" in d[name]: del d[name]["Sequence"]
    if "Label name" in d[name]: del d[name]["Label name"]
    if "Image" in d[name]: del d[name]["Image"]
    if "Mask" in d[name]: del d[name]["Mask"]
    if "src" in d[name]: del d[name]["src"]

    if "Image_right" in d[name]: del d[name]["Image_right"]
    if "Mask_right" in d[name]: del d[name]["Mask_right"]
    if "Label name_right" in d[name]: del d[name]["Label name_right"]
    if "Sequence_right" in d[name]: del d[name]["Sequence_right"]
    if "bbox_right" in d[name]: del d[name]["bbox_right"]
    if "src_right" in d[name]: del d[name]["src_right"]
    seq = ob["Sequence"]
    label = ob["Label name"].lower() if ob["Label name"] is not None else None
    d[name][f"{seq} {label} image"] = ob["Image"]
    d[name][f"{seq} {label} mask"] = ob["Mask"]
    d[name][f"{seq} {label} bbox"] = ob["bbox"]

    seq = ob["Sequence_right"]
    label = ob["Label name_right"].lower() if ob["Label name_right"] is not None else None
    d[name][f"{seq} hga-{label} image"] = ob["Image"]
    d[name][f"{seq} hga-{label} mask"] = ob["Mask"]
    d[name][f"{seq} hga-{label} bbox"] = ob["bbox"]

d["Patient-001/week-000-1"]

{'Patient': 'Patient-001',
 'Survival time (weeks)': '89',
 'Sex': 'female',
 'Age at surgery (years)': 57,
 'IDH (WT: wild type)': 'WT',
 'IDH method': 'BES',
 'MGMT qualitative': 'methylated',
 'MGMT quantitative': 'na',
 'Date': 'week-000-1',
 'LessThan3Months': None,
 'NonMeasurableLesions': None,
 'RANO': 'Pre-Op',
 'bbox': [51, 39, 87, 35, 41, 45],
 'CT1 necrosis image': 'Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz',
 'CT1 necrosis mask': 'Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz',
 'CT1 necrosis bbox': [51, 39, 87, 35, 41, 45],
 'CT1 hga-non-enhancing image': 'Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz',
 'CT1 hga-non-enhancing mask': 'Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz',
 'CT1 hga-non-enhancing bbox': [39, 32, 77, 51, 61, 62],
 'CT1 hga-contrast-enhancing image': 'Patient-001/week-000-1/DeepBra

In [190]:
df = pl.DataFrame(list(d.values()))
df

Patient,Survival time (weeks),Sex,Age at surgery (years),IDH (WT: wild type),IDH method,MGMT qualitative,MGMT quantitative,Date,LessThan3Months,NonMeasurableLesions,RANO,bbox,CT1 necrosis image,CT1 necrosis mask,CT1 necrosis bbox,CT1 hga-non-enhancing image,CT1 hga-non-enhancing mask,CT1 hga-non-enhancing bbox,CT1 hga-contrast-enhancing image,CT1 hga-contrast-enhancing mask,CT1 hga-contrast-enhancing bbox,T1 hga-non-enhancing image,T1 hga-non-enhancing mask,T1 hga-non-enhancing bbox,T1 hga-contrast-enhancing image,T1 hga-contrast-enhancing mask,T1 hga-contrast-enhancing bbox,T2 hga-non-enhancing image,T2 hga-non-enhancing mask,T2 hga-non-enhancing bbox,T2 hga-contrast-enhancing image,T2 hga-contrast-enhancing mask,T2 hga-contrast-enhancing bbox,FLAIR hga-non-enhancing image,FLAIR hga-non-enhancing mask,FLAIR hga-non-enhancing bbox,FLAIR hga-contrast-enhancing image,FLAIR hga-contrast-enhancing mask,FLAIR hga-contrast-enhancing bbox,CT1 contrast-enhancing image,CT1 contrast-enhancing mask,CT1 contrast-enhancing bbox,CT1 edema image,CT1 edema mask,CT1 edema bbox,T1 necrosis image,T1 necrosis mask,T1 necrosis bbox,T1 contrast-enhancing image,T1 contrast-enhancing mask,T1 contrast-enhancing bbox,T1 edema image,T1 edema mask,T1 edema bbox,T2 necrosis image,T2 necrosis mask,T2 necrosis bbox,T2 contrast-enhancing image,T2 contrast-enhancing mask,T2 contrast-enhancing bbox,T2 edema image,T2 edema mask,T2 edema bbox,FLAIR necrosis image,FLAIR necrosis mask,FLAIR necrosis bbox,FLAIR contrast-enhancing image,FLAIR contrast-enhancing mask,FLAIR contrast-enhancing bbox,FLAIR edema image,FLAIR edema mask,FLAIR edema bbox
str,str,str,i64,str,str,str,str,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64]
"""Patient-001""","""89""","""female""",57,"""WT""","""BES""","""methylated""","""na""","""week-000-1""",null,null,"""Pre-Op""","[51, 39, … 45]","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/ct1_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","[51, 39, … 45]","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","[39, 32, … 62]","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","[39, 32, … 62]","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","[39, 32, … 62]","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","[39, 32, … 62]","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","[39, 32, … 62]","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","[39, 32, … 62]","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz""","[39, 32, … 62]","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz""","""Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmenta

Deepbratumia: necrosis, DBT contrast-enhancing, edema

HD-GLIO-AUTO: non-enhancing, HGA contrast-enhancing

In [193]:
inputs = ["FLAIR necrosis image", "T1 necrosis image", "Age at surgery (years)", "Sex"]
targets = ["FLAIR necrosis mask", "FLAIR necrosis bbox"]

df.select(inputs+targets).to_dicts()

[{'FLAIR necrosis image': 'Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz',
  'T1 necrosis image': 'Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/t1_skull_strip.nii.gz',
  'Age at surgery (years)': 57,
  'Sex': 'female',
  'FLAIR necrosis mask': 'Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz',
  'FLAIR necrosis bbox': [51, 39, 87, 35, 41, 45]},
 {'FLAIR necrosis image': 'Patient-001/week-000-2/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz',
  'T1 necrosis image': 'Patient-001/week-000-2/DeepBraTumIA-segmentation/atlas/skull_strip/t1_skull_strip.nii.gz',
  'Age at surgery (years)': 57,
  'Sex': 'female',
  'FLAIR necrosis mask': 'Patient-001/week-000-2/DeepBraTumIA-segmentation/atlas/segmentation/seg_mask.nii.gz',
  'FLAIR necrosis bbox': [51, 48, 97, 36, 38, 39]},
 {'FLAIR necrosis image': 'Patient-001/week-044/DeepBraTumIA-segmentation/atlas/skull_strip/flair_

## всё вместе, функция для создания DS

In [194]:
def load(cols:list[str]):
    # 1. Чтение csv и формирования DataFrame
    demographics_pathology = pl.read_csv(f"{path}/LUMIERE-Demographics_Pathology.csv", infer_schema_length=10000)


    def rano_norm(x: str):
        if x == "Post-Op ": return "Post-Op"
        elif x == "Post-Op/PD": return "PD"
        else: return x

    expert_rating = (
        pl.read_csv(f"{path}/LUMIERE-ExpertRating-v202211.csv", infer_schema_length=10000)
        .with_columns(RANO=pl.col(RANO).map_elements(rano_norm))
        .drop(RANO, RANO_rationale))

    deepbratumia = (
        pl.read_csv(
            f"{path}/LUMIERE-pyradiomics-deepbratumia-features.csv",
            infer_schema_length=10000,)
        .select("Patient","Time point","Image","Mask","Label name","Sequence","diagnostics_Mask-original_BoundingBox",)
        .with_columns(
            bbox=pl.col("diagnostics_Mask-original_BoundingBox").map_elements(lambda x: [int(i) for i in x[1:-1].split(", ")]),
            src=pl.lit("Deepbratumia"),)
        .drop("diagnostics_Mask-original_BoundingBox")
        .rename({"Time point": "Date"}))

    hdglioauto = (
        pl.read_csv(
            f"{path}/LUMIERE-pyradiomics-hdglioauto-features.csv", infer_schema_length=10000)
        .select("Patient","Time point","Image","Mask","Label name","Sequence","diagnostics_Mask-original_BoundingBox",)
        .with_columns(
            bbox=pl.col("diagnostics_Mask-original_BoundingBox").map_elements(lambda x: [int(i) for i in x[1:-1].split(", ")]),
            src=pl.lit("HD-GLIO-AUTO"),)
        .drop("diagnostics_Mask-original_BoundingBox")
        .rename({"Time point": "Date"}))

    combined = (
        demographics_pathology.join(expert_rating, on="Patient", how="outer_coalesce")
        .join(deepbratumia, on=["Patient", "Date"], how="outer_coalesce")
        .join(hdglioauto, on=["Patient", "Date"], how="outer_coalesce"))

    # 2. Аггрегация колонок
    d = {}
    for ob in combined.rows(named=True):
        name = f'{ob["Patient"]}/{ob["Date"]}'
        if name not in d: d[name] = ob.copy()
        if "Sequence" in d[name]: del d[name]["Sequence"]
        if "Label name" in d[name]: del d[name]["Label name"]
        if "Image" in d[name]: del d[name]["Image"]
        if "Mask" in d[name]: del d[name]["Mask"]
        if "src" in d[name]: del d[name]["src"]

        if "Image_right" in d[name]: del d[name]["Image_right"]
        if "Mask_right" in d[name]: del d[name]["Mask_right"]
        if "Label name_right" in d[name]: del d[name]["Label name_right"]
        if "Sequence_right" in d[name]: del d[name]["Sequence_right"]
        if "bbox_right" in d[name]: del d[name]["bbox_right"]
        if "src_right" in d[name]: del d[name]["src_right"]
        seq = ob["Sequence"]
        label = ob["Label name"].lower() if ob["Label name"] is not None else None
        d[name][f"{seq} {label} image"] = ob["Image"]
        d[name][f"{seq} {label} mask"] = ob["Mask"]
        d[name][f"{seq} {label} bbox"] = ob["bbox"]

        seq = ob["Sequence_right"]
        label = ob["Label name_right"].lower() if ob["Label name_right"] is not None else None
        d[name][f"{seq} hga-{label} image"] = ob["Image"]
        d[name][f"{seq} hga-{label} mask"] = ob["Mask"]
        d[name][f"{seq} hga-{label} bbox"] = ob["bbox"]

    df = pl.DataFrame(list(d.values()))
    return df.select(cols).to_dicts()

In [195]:
load(["FLAIR necrosis image", "T1 necrosis image", "Age at surgery (years)", "Sex"])

[{'FLAIR necrosis image': 'Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz',
  'T1 necrosis image': 'Patient-001/week-000-1/DeepBraTumIA-segmentation/atlas/skull_strip/t1_skull_strip.nii.gz',
  'Age at surgery (years)': 57,
  'Sex': 'female'},
 {'FLAIR necrosis image': 'Patient-001/week-000-2/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz',
  'T1 necrosis image': 'Patient-001/week-000-2/DeepBraTumIA-segmentation/atlas/skull_strip/t1_skull_strip.nii.gz',
  'Age at surgery (years)': 57,
  'Sex': 'female'},
 {'FLAIR necrosis image': 'Patient-001/week-044/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz',
  'T1 necrosis image': 'Patient-001/week-044/DeepBraTumIA-segmentation/atlas/skull_strip/t1_skull_strip.nii.gz',
  'Age at surgery (years)': 57,
  'Sex': 'female'},
 {'FLAIR necrosis image': 'Patient-001/week-056/DeepBraTumIA-segmentation/atlas/skull_strip/flair_skull_strip.nii.gz',
  'T1 necrosis image'

In [1]:
import torch
from glio.datasets.LUMIERE import LUMIERE
from glio.data import DSToTarget
from glio.jupyter_tools import show_slices_arr
from glio.loaders import nifti
lum = LUMIERE()

ds = DSToTarget()
ds.add_samples(lum.get_cols(["Age at surgery (years)", "Sex", "T1 necrosis image", "FLAIR necrosis image", "FLAIR necrosis mask", "FLAIR necrosis bbox"]),
               loader = lambda x: dict(age=x["Age at surgery (years)"],
                                       sex=x["Sex"],
                                       img=torch.stack((nifti.read(x["T1 necrosis image"]), nifti.read(x["FLAIR necrosis image"]))),
                                       bbox = x["FLAIR necrosis bbox"]),
               transform_sample=lambda x: x["img"],
               transform_target=lambda x: x["bbox"])
print(len(ds))
show_slices_arr(ds[0][0])

497


interactive(children=(Checkbox(value=False, description='color'), Text(value='0 1 2 3', description='permute')…

<function glio.jupyter_tools.show_slices_arr.<locals>.f(color, permute: str, **kwargs)>

In [2]:
torch.set_printoptions(precision=10)
t1_mean_std = ds.get_mean_std(500, 16)
print(t1_mean_std)

(tensor([21.6240825653, 17.5688686371]), tensor([48.3810195923, 38.1297569275]))811, 135.5724639893])


In [ ]:
from glio.python_tools import RepeatCacheIterator

r = RepeatCacheIterator([1,2,3,4,5], 3)

for i in r: print(i)